# Amazon Bedrock AgentCore 런타임(Runtime)에서 Amazon Bedrock 모델을 사용한 CrewAI 멀티 에이전트(Multi-Agent) 크루 호스팅

## 개요

이 튜토리얼에서는 Amazon Bedrock AgentCore 런타임(Runtime)을 사용하여 기존 멀티 에이전트(Multi-Agent) 크루를 호스팅하는 방법을 배웁니다.

Amazon Bedrock 모델을 사용한 CrewAI 예제에 초점을 맞출 것입니다. Amazon Bedrock 모델을 사용한 Strands 에이전트(Agent)는 [여기](../01-strands-with-bedrock-model)를, OpenAI 모델을 사용한 Strands 에이전트(Agent)는 [여기](../03-strands-with-openai-model)를 확인하세요.


### 튜토리얼 세부사항

| 정보 | 세부사항 |
|:--------------------|:-----------------------------------------------------------------------------|
| 튜토리얼 유형 | 대화형 |
| 에이전트(Agent) 유형 | 멀티 에이전트(Multi-Agent) 크루 |
| 에이전틱 프레임워크(Agentic Framework) | CrewAI |
| LLM 모델 | Anthropic Claude Haiku 4.5 |
| 튜토리얼 구성 요소 | AgentCore 런타임(Runtime)에 에이전트(Agent) 호스팅. CrewAI와 Amazon Bedrock 모델 사용 |
| 튜토리얼 분야 | 범용 |
| 예제 복잡도 | 쉬움 |
| 사용 SDK | Amazon BedrockAgentCore Python SDK 및 boto3 |

### 튜토리얼 아키텍처

이 튜토리얼에서는 기존 멀티 에이전트(Multi-Agent) 크루를 AgentCore 런타임(Runtime)에 배포(Deployment)하는 방법을 설명합니다.

데모 목적으로 Amazon Bedrock 모델을 사용하는 CrewAI 크루를 사용할 것입니다.

예제에서는 연구원(researcher)과 분석가(analyst) 두 에이전트(Agent)가 있는 리서치 크루를 사용합니다.
<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="60%"/>
</div>


### 튜토리얼 주요 기능

* Amazon Bedrock AgentCore 런타임(Runtime)에 에이전트(Agent) 호스팅
* Amazon Bedrock 모델 사용
* CrewAI 사용

## 사전 요구사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* uv 패키지 관리자
* AWS 자격 증명
* Docker 실행 중

추가로 몇 가지 종속성을 설치해야 합니다:
* Amazon Bedrock AgentCore SDK
* CrewAI
* Langchain community 패키지
* Duckduckgo 검색

필요한 모든 종속성을 pyproject.toml 파일에 패키징했으므로 편리하게 설치할 수 있습니다.

In [ ]:
!uv sync --active --force-reinstall

## Creating your multi-agent crew and experimenting locally

Before we deploy our agents to AgentCore Runtime, let's develop and run them locally for experimentation purposes.

In this guide, we’ll walk through creating a research crew that will help us research and analyze a topic, then create a comprehensive report. This practical example demonstrates how AI agents can collaborate to accomplish complex tasks. The example is adapted from a [getting started guide](https://docs.crewai.com/en/guides/crews/first-crew) provided directly by CrewAI.

The local architecture looks as following:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="60%"/>
</div>


### Defining agents, tasks, crew

We will first create the artifacts defining a local CrewAI agent, including: 
* agents.yaml, defining the two agents involved in our crew
* tasks.yaml, defining the tasks to be executed by the agents in our crew
* crew.py, defining our crew consisting of agents working on tasks as defined
* main.py, our local entrypoint kicking off the crew run

In [ ]:
import os
os.makedirs('research_crew/config', exist_ok=True)

In [ ]:
%%writefile research_crew/config/agents.yaml
researcher:
  role: >
    Senior Research Specialist for {topic}
  goal: >
    Find comprehensive and accurate information about {topic}
    with a focus on recent developments and key insights
  backstory: >
    You are an experienced research specialist with a talent for
    finding relevant information from various sources. You excel at
    organizing information in a clear and structured manner, making
    complex topics accessible to others.
  llm: bedrock/global.anthropic.claude-haiku-4-5-20251001-v1:0

analyst:
  role: >
    Data Analyst and Report Writer for {topic}
  goal: >
    Analyze research findings and create a comprehensive, well-structured
    report that presents insights in a clear and engaging way
  backstory: >
    You are a skilled analyst with a background in data interpretation
    and technical writing. You have a talent for identifying patterns
    and extracting meaningful insights from research data, then
    communicating those insights effectively through well-crafted reports.
  llm: bedrock/global.anthropic.claude-haiku-4-5-20251001-v1:0

In [ ]:
%%writefile research_crew/config/tasks.yaml
research_task:
  description: >
    Conduct thorough research on {topic}. Focus on:
    1. Key concepts and definitions
    2. Historical development and recent trends
    3. Major challenges and opportunities
    4. Notable applications or case studies
    5. Future outlook and potential developments

    Make sure to organize your findings in a structured format with clear sections.
  expected_output: >
    A comprehensive research document with well-organized sections covering
    all the requested aspects of {topic}. Include specific facts, figures,
    and examples where relevant.
  agent: researcher

analysis_task:
  description: >
    Analyze the research findings and create a comprehensive report on {topic}.
    Your report should:
    1. State the topic and begin with an executive summary
    2. Include all key information from the research
    3. Provide insightful analysis of trends and patterns
    4. Offer recommendations or future considerations
    5. Be formatted in a professional, easy-to-read style with clear headings
  expected_output: >
    A polished, professional report on {topic} that presents the research
    findings with added analysis and insights. The report should be well-structured
    with an executive summary, main sections, and conclusion.
  agent: analyst
  context:
    - research_task

In [ ]:
%%writefile research_crew/crew.py
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task
from crewai.agents.agent_builder.base_agent import BaseAgent
from typing import List
from langchain_community.tools import DuckDuckGoSearchRun
from crewai.tools import BaseTool
from crewai_tools import SerperDevTool
from pydantic import Field


class SearchTool(BaseTool):
     name: str = "Search"
     description: str = "Useful for searching the web for information."
     search: DuckDuckGoSearchRun = Field(default_factory=DuckDuckGoSearchRun)

     def _run(self, query: str) -> str:
         """Execute the search query and return results"""
         try:
             return self.search.invoke(query)
         except Exception as e:
             return f"Error performing search: {str(e)}"

@CrewBase
class ResearchCrew():
    """Research crew for comprehensive topic analysis and reporting"""

    agents: List[BaseAgent]
    tasks: List[Task]

    @agent
    def researcher(self) -> Agent:
        return Agent(
            config=self.agents_config['researcher'], # type: ignore[index]
            verbose=True,
            tools=[
                #SerperDevTool()
                SearchTool()
                ]
        )

    @agent
    def analyst(self) -> Agent:
        return Agent(
            config=self.agents_config['analyst'], # type: ignore[index]
            verbose=True
        )

    @task
    def research_task(self) -> Task:
        return Task(
            config=self.tasks_config['research_task'] # type: ignore[index]
        )

    @task
    def analysis_task(self) -> Task:
        return Task(
            config=self.tasks_config['analysis_task'], # type: ignore[index]
            #output_file='output/report.md'
        )

    @crew
    def crew(self) -> Crew:
        """Creates the research crew"""
        return Crew(
            agents=self.agents,
            tasks=self.tasks,
            process=Process.sequential,
            verbose=True,
        )

In [ ]:
%%writefile research_crew/main.py
import os
from research_crew.crew import ResearchCrew

# Create output directory if it doesn't exist
os.makedirs('output', exist_ok=True)

def run():
    """
    Run the research crew.
    """
    inputs = {
        'topic': 'Artificial Intelligence in Healthcare'
    }

    # Create and run the crew
    result = ResearchCrew().crew().kickoff(inputs=inputs)

    # Print the result
    print("\n\n=== FINAL REPORT ===\n\n")
    print(result.raw)


if __name__ == "__main__":
    run()

## 멀티 에이전트(Multi-Agent) 크루 생성 및 로컬에서 실험하기

에이전트(Agent)를 AgentCore 런타임(Runtime)에 배포(Deployment)하기 전에 실험 목적으로 로컬에서 개발하고 실행해 보겠습니다.

이 가이드에서는 주제를 연구하고 분석한 다음 종합 보고서를 작성하는 리서치 크루를 만드는 과정을 살펴봅니다. 이 실제 예제는 AI 에이전트(Agent)들이 어떻게 협력하여 복잡한 작업을 수행할 수 있는지 보여줍니다. 이 예제는 CrewAI에서 직접 제공하는 [시작 가이드](https://docs.crewai.com/en/guides/crews/first-crew)에서 수정되었습니다.

로컬 아키텍처는 다음과 같습니다:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="60%"/>
</div>


### 에이전트(Agent), 태스크, 크루 정의

먼저 로컬 CrewAI 에이전트(Agent)를 정의하는 아티팩트를 생성합니다:
* agents.yaml, 크루에 참여하는 두 에이전트(Agent) 정의
* tasks.yaml, 크루의 에이전트(Agent)들이 수행할 태스크 정의
* crew.py, 정의된 대로 태스크를 수행하는 에이전트(Agent)로 구성된 크루 정의
* main.py, 크루 실행을 시작하는 로컬 진입점

In [ ]:
!crewai run

## Deploying multi-agent crew to Amazon Bedrock AgentCore

For production-grade agentic applications we will need to run our crew in the cloud. Therefor we will deploy our crew to Amazon Bedrock AgentCore. 

The architecture here will look as following:

<div style="text-align:left">
     <img src="images/architecture_local.png" width="60%"/>
</div>

Deploying the crew to AgentCore takes the following steps: 

### Remote entrypoint

First, we create a remote entrypoint. With AgentCore Runtime, we will decorate the invocation part of our agent with the @app.entrypoint decorator and have it as the entry point for our runtime. This also involves: 
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`

### What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards                                                                                                                                                                        

In [ ]:
%%writefile research_crew/research_crew.py
import os
from research_crew.crew import ResearchCrew

# ---------- Agentcore imports --------------------
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()
#------------------------------------------------


@app.entrypoint
def agent_invocation(payload, context):
    """Handler for agent invocation"""
    print(f'Payload: {payload}')
    try: 
        # Extract user message from payload with default
        user_message = payload.get("prompt", "Artificial Intelligence in Healthcare")
        print(f"Processing topic: {user_message}")
        
        # Create crew instance and run synchronously
        research_crew_instance = ResearchCrew()
        crew = research_crew_instance.crew()
        
        # Use synchronous kickoff instead of async - this avoids all event loop issues
        result = crew.kickoff(inputs={'topic': user_message})

        print("Context:\n-------\n", context)
        print("Result Raw:\n*******\n", result.raw)
        
        # Safely access json_dict if it exists
        if hasattr(result, 'json_dict'):
            print("Result JSON:\n*******\n", result.json_dict)
        
        return {"result": result.raw}
        
    except Exception as e:
        print(f'Exception occurred: {e}')
        return {"error": f"An error occurred: {str(e)}"}

if __name__ == "__main__":
    app.run()

### Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCode Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

#### Configure AgentCore Runtime deployment

First we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

AgentCore configure is required to generate a Dockerfile holding a blueprint for the Docker container the workload will be running in and a .bedrock_agentcore.yaml holding the agentic workload's configuration. During the configure step, your docker file will be generated based on your application code.

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "research_crew_getting_started"
response = agentcore_runtime.configure(
    entrypoint="research_crew/research_crew.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    region=region,
    agent_name=agent_name
)
response

#### Launching agent to AgentCore Runtime: deploying the remote agentic workload

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime. AgentCore launch will then deploy the agentic workload to the cloud. This includes creating a Docker image and pushing it to ECR, as well as getting an endpoint ready for usage.


<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()

#### Checking for the AgentCore Runtime Status

Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it. Since this is a long running agent we are overwriting the default `retries`, `connect_timout`and `read_timeout`.

<div style="text-align:left">
    <img src="images/invoke.png" width=85%"/>
</div>

In [ ]:
from botocore.config import Config

# Configure retries and timeout
config = Config(
    retries={
        'max_attempts': 10,  # Increase max retries to 10 (default is 4)
        'mode': 'adaptive'   # Options: 'legacy', 'standard', 'adaptive'
    },
    connect_timeout=600,      # Connection timeout in seconds (default is 60)
    read_timeout=3000         # Read timeout in seconds (default is 60)
)

In [ ]:
import boto3
import json
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region,
    config=config
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "Artificial Intelligence in Healthcare"})
)

response_body = boto3_response['response'].read()
response_data = json.loads(response_body)
response_data

### 로컬에서 크루 호출(Invocation)

마지막으로 CrewAI CLI를 사용하여 로컬에서 크루를 시작할 수 있습니다. 또는 로컬 진입점 main.py를 직접 실행할 수도 있습니다. 이 작업은 몇 분이 걸릴 수 있습니다.

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

## Congratulations!